# HW07: Deep Learning (due 16th November)

In this homework, you will replicate the heterogenous treatment effect exercise from last week's homework using a deep learning model instead of a machine learning one. 

For those who did not complete this exercise, you will investigate the effect of case management on mental health outcomes. These data come from a randomized control trial where patients were assigned to *intensive* or *standard* case management. In this context, the treatment is being assigned to the **intensive** case management while patients assigned to the **standard** case management belong to the control group.

We will investigate characteristics of individuals who are most and least responsive to the treatment, i.e., to being assigned to the intensive case management. 

In [9]:
import pandas as pd
from tensorflow import keras

In [3]:
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_stata('http://www.homepages.ucl.ac.uk/~rmjwiww/stata/missing/uk500.dta')
df = df.dropna()
pd.read_stata('http://www.homepages.ucl.ac.uk/~rmjwiww/stata/missing/uk500.dta', iterator=True).variable_labels()

{'trialid': 'Trial ID',
 'centreid': 'Trial centre',
 'status': 'Patient status at baseline',
 'age': 'Age in years at baseline',
 'sex': 'Sex',
 'afcarib': 'Ethnic group',
 'ocfabth': "Father's social class at birth",
 'chron1l': 'Months since onset of psychosis, logged',
 'hos94': 'Days in hospital for psychiatric reasons: 2 years before baseline',
 'cprs94': 'Psychopathology at baseline (CPRS)',
 'das94': 'Disability at baseline (DAS)',
 'sat94': '(Dis)satisfaction with services at baseline',
 'rand': 'Randomised group',
 'hos96': 'Days in hospital for psychiatric reasons: 2 years after baseline',
 'cprs96': 'Psychopathology at 2 years (CPRS)',
 'sat96': '(Dis)satisfaction with services at 2 years'}

In [4]:
df.head()

,trialid,centreid,status,age,sex,afcarib,ocfabth,chron1l,hos94,cprs94,das94,sat94,rand,hos96,cprs96,sat96
1,107.0,St George's,Out-patient,27.0,male,Other,A,3.178054,80.0,4.0,0.285714,18.0,Intensive case management,27.0,3.0,22.000
2,222005.0,St Mary's,In hospital,41.0,male,Other,D,4.521789,240.0,6.0,0.750000,15.0,Intensive case management,15.0,13.0,9.000
3,222018.0,St Mary's,In hospital,25.0,male,Other,C2,4.094345,48.0,12.0,0.125000,18.0,Intensive case management,263.0,6.0,21.375
5,312015.0,King's,Out-patient,31.0,female,Other,A,4.787492,60.0,28.0,2.375000,20.0,Intensive case management,45.0,19.0,17.000
6,221023.0,St Mary's,In hospital,35.0,male,Afro-Caribbean,C2,4.430817,60.0,25.0,1.571428,24.0,Intensive case management,58.0,27.0,19.125


The treatment variable is $rand$, the post-treatment outcomes are $hos96$, $cprs96$ and $sat96$.

In [5]:
treatvar = 'rand'
df[treatvar].value_counts()

Intensive case management    130
Standard case management     116
Name: rand, dtype: int64

In [57]:
outcomes = ['sat96', 'hos96', 'cprs96']
display(df[outcomes].describe())
outcome = outcomes[2]

,sat96,hos96,cprs96
count,246.000000,246.000000,246.000000
mean,17.271341,65.500000,17.790587
std,4.723009,104.046722,14.090911
min,9.000000,0.000000,0.000000
25%,14.000000,0.000000,7.000000
50%,17.000000,15.000000,15.000000
75%,20.187500,93.500000,26.000000
max,32.000000,692.000000,71.000000


In addition to these variables we need a set of covariates that we want to use to identify individuals who are most and least responsive to treatment. We also encode categorical covariates and prepare them for the ML model.

In [58]:
#Encoding Categorical covariates and preparing the data for tensorflow
covariates = ['status', 'sex', 'sat94', 'ocfabth', 'hos94', 'das94', 'cprs94', 'age', 'afcarib']
covariates_cat = ['status', 'sex', 'ocfabth', 'afcarib']

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df[covariates_cat] = encoder.fit_transform(df[covariates_cat])
df[covariates] = df[covariates].astype('float32')
df[covariates].head()

,status,sex,sat94,ocfabth,hos94,das94,cprs94,age,afcarib
1,1.0,1.0,18.0,0.0,80.0,0.285714,4.0,27.0,1.0
2,0.0,1.0,15.0,4.0,240.0,0.750000,6.0,41.0,1.0
3,0.0,1.0,18.0,3.0,48.0,0.125000,12.0,25.0,1.0
5,1.0,0.0,20.0,0.0,60.0,2.375000,28.0,31.0,1.0
6,0.0,1.0,24.0,3.0,60.0,1.571428,25.0,35.0,0.0


In [59]:
# Subset the dataset by treatment and control
# Within each sample, create a training, a test and a validation set
from sklearn.model_selection import train_test_split

df_treat = df[df[treatvar] == "Intensive case management"]
# create a training, validation and a test set for the treatment group
X_t_train, X_t_test, y_t_train, y_t_test = train_test_split(df_treat[covariates], df_treat[outcome], test_size=0.2, random_state=42)
X_t_train, X_t_val, y_t_train, y_t_val = train_test_split(X_t_train, y_t_train, test_size=0.2, random_state=42)

df_control = df[df[treatvar] == "Standard case management"]
# create a training, validation and a test set for the control group
X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(df_control[covariates], df_control[outcome], test_size=0.2, random_state=42)
X_c_train, X_c_val, y_c_train, y_c_val = train_test_split(X_c_train, y_c_train, test_size=0.2, random_state=42)


In [60]:
##choose one of the three outcomes to analyze
## build two MLP (multilayer perceptron) models to predict the outcome based on the covariates
## the first model should be trained on the treated sample, while the second on the control

# the two MLP models should have with at least 2 hidden layers, ReLU activation, batch normalization, dropout

model_treat = keras.models.Sequential()
model_control = keras.models.Sequential()

layers = [
    keras.layers.Dense(6, input_dim=9, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(6, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="softmax")
]
for layer in layers:
    model_treat.add(layer)
    model_control.add(layer)

In [61]:
# compile the models

model_treat.compile(loss="mean_squared_error", optimizer="adam")
model_control.compile(loss="mean_squared_error", optimizer="adam")

In [62]:
# fit separate models on the treatment dataset and control dataset
# use early stopping

early_stopping_cb = keras.callbacks.EarlyStopping(monitor="loss", patience=10, restore_best_weights=True)

model_treat.fit(X_t_train, y_t_train, epochs=100, validation_data=(X_t_val, y_t_val), callbacks=[early_stopping_cb])
model_control.fit(X_c_train, y_c_train, epochs=100, validation_data=(X_c_val, y_c_val), callbacks=[early_stopping_cb])

Epoch 1/100
3/3 [==============================] - 1s 111ms/step - loss: 534.2866 - val_loss: 415.9524
Epoch 2/100
3/3 [==============================] - 0s 14ms/step - loss: 534.2866 - val_loss: 415.9524
Epoch 3/100
3/3 [==============================] - 0s 16ms/step - loss: 534.2866 - val_loss: 415.9524
Epoch 4/100
3/3 [==============================] - 0s 18ms/step - loss: 534.2865 - val_loss: 415.9524
Epoch 5/100
3/3 [==============================] - 0s 16ms/step - loss: 534.2866 - val_loss: 415.9524
Epoch 6/100
3/3 [==============================] - 0s 14ms/step - loss: 534.2866 - val_loss: 415.9524
Epoch 7/100
3/3 [==============================] - 0s 16ms/step - loss: 534.2866 - val_loss: 415.9524
Epoch 8/100
3/3 [==============================] - 0s 16ms/step - loss: 534.2865 - val_loss: 415.9524
Epoch 9/100
3/3 [==============================] - 0s 17ms/step - loss: 534.2866 - val_loss: 415.9524
Epoch 10/100
3/3 [==============================] - 0s 18ms/step - loss: 534.2866

In [63]:
##get predicted outcomes using the combined test sets for both models 

test_set = X_t_test.append(X_c_test)
yhat_treat = model_treat.predict(test_set)
yhat_control = model_control.predict(test_set)

2/2 [==============================] - 0s 3ms/step


/var/folders/r0/g49v1kk56jb9__0w14vks_6m0000gn/T/ipykernel_78560/740953963.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_set = X_t_test.append(X_c_test)


2/2 [==============================] - 0s 2ms/step


In [64]:
##find the three individuals in the test set that are most and least responsive to the treatment
##namely the three individuals for who the treatment effect is larger and those for who it is smaller

test_set['treat_effect'] = yhat_treat - yhat_control
print("Three individuals with the largest treatment effect:")
display(test_set.sort_values(by='treat_effect', ascending=False).head(3))
print("Three individuals with the smallest treatment effect:")
display(test_set.sort_values(by='treat_effect', ascending=True).head(3))

Three individuals with the largest treatment effect:


,status,sex,sat94,ocfabth,hos94,das94,cprs94,age,afcarib,treat_effect
182,1.0,1.0,22.0,1.0,201.0,1.428571,25.0,23.0,0.0,0.0
135,0.0,0.0,24.0,1.0,150.0,1.000000,19.0,63.0,1.0,0.0
27,0.0,1.0,26.0,3.0,35.0,2.166666,26.0,40.0,1.0,0.0


Three individuals with the smallest treatment effect:


,status,sex,sat94,ocfabth,hos94,das94,cprs94,age,afcarib,treat_effect
182,1.0,1.0,22.0,1.0,201.0,1.428571,25.0,23.0,0.0,0.0
27,0.0,1.0,26.0,3.0,35.0,2.166666,26.0,40.0,1.0,0.0
209,0.0,1.0,20.0,3.0,14.0,0.833333,11.0,50.0,1.0,0.0


Weirdly enough, two of the three individuals with the largest treatment effects are also the ones with the smallest treatment effect, suggesting that something was wrong here. Otherwise there is nothing special that is common among these.